# EEG - Flow

## 7. ICA apply
This steps requires that a comparison has been done between two reviewers.
Both person will either select the best selection or make a new one together, then will rename the file to match the step 5 convention.

Last edit: 21.06.2023 19:25

In [2]:
import multiprocessing as mp
import time
from itertools import product

from mne import set_log_level as set_log_level_mne

from eeg_flow import set_log_level
from eeg_flow.utils import parallel
from eeg_flow.tasks import apply_ica

2024-05-10 11:08:53,693 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-05-10 11:08:53,695 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [3]:
set_log_level_mne("WARNING")
set_log_level("INFO")

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [4]:
"P16-G4", "P18-G6"
PARTICIPANTS_WITH_GROUPS: list[str] = ["P02-G2","P05-G2","P07-G5","P08-G6","P11-G1","P12-G2","P13-G3","P14-G3","P15-G4",
                                      "P18-G6","P19-G6","P21-G7","P22-G7","P23-G7","P25-G1","P26-G2","P27-G8","P30-G4","P31-G5",

SyntaxError: incomplete input (2736383380.py, line 3)

In [6]:
PARTICIPANTS_WITH_GROUPS: list[str] = ["P32-G6"]  # list of str "Pxx-Gy", e.g. ["P02-G2"]

TASKS: list[str]                    = ["oddball","UT"]  # ["oddball"], ["UT"] or ["oddball", "UT"]
RUNS: list[int]                     = [1,2]  # [1], [2] or [1, 2]

inputs = [
    (t[0].split("-") + list(t[1:]))
    for t in product(PARTICIPANTS_WITH_GROUPS, TASKS, RUNS)
]
print(inputs)

[['P32', 'G6', 'oddball', 1], ['P32', 'G6', 'oddball', 2], ['P32', 'G6', 'UT', 1], ['P32', 'G6', 'UT', 2]]


The variable `inputs` contains is a list of list. Each sublist defines one file by its participant, group, task and run attribute. Each sublist is one set of input variable for `apply_ica` which will be picked up by a worker (process) and executed. For each execution, the created derivatives are:
- Preprocessed raw recording (`_step7_preprocessed_raw.fif`)

This step ends with a visual inspection of the raw recording during which `BAD` segments annotations should be edited based on what was well corrected and what was not.

In [7]:
%%time
current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

assert len(inputs) != 0  # sanity-check
# each job will need 1 threads and mp.cpu_count() usually returns
# the number of threads (usually 2 per core), thus let's spawn 2
# process per core maximum.
n_jobs = min(len(inputs), (mp.cpu_count() - 2))
parallel(apply_ica, n_jobs, inputs)

Start time 11:34:49


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:35<00:00,  8.97s/it]

CPU times: total: 78.1 ms
Wall time: 35.9 s


[None, None, None, None]